**Introducción**

En el presente proyecto de análisis financiero, se utilizará Python en la plataforma Jupyter Colab para llevar a cabo un estudio detallado de las empresas que cotizan en el IPSA (Índice de Precios Selectivo de Acciones). El objetivo principal es analizar los estados financieros de estas compañías y agruparlas en función de sus similitudes, aplicando técnicas de Machine Learning.

La finalidad del proyecto es desarrollar un código eficiente y automatizado que permita identificar patrones de comportamiento financiero entre las empresas, independientemente del sector económico al que pertenezcan. Esto facilitará la detección de grupos con dinámicas similares, proporcionando una herramienta útil para inversionistas, analistas y otros actores del mercado financiero.

In [ ]:
#importamos librerias a utilizar
import pandas as pd
from datetime import datetime
import pandas_datareader as web
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
%matplotlib inline

# Sección nueva

In [ ]:
print("intentando push")

intentando push


In [ ]:
!pip install yfinance

import yfinance as yf

Revisamos la informacion que podemos obtener de yfinance

In [ ]:
# Define the date range for the data
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 8, 30)

# Fetch Google stock data from Yahoo Finance using yfinance
google_data = yf.download("BSANTANDER.SN", start=start_date, end=end_date)
# Display the first few rows of the fetched data
print(google_data.head())

[*********************100%***********************]  1 of 1 completed

Price          Adj Close         Close          High           Low  \
Ticker     BSANTANDER.SN BSANTANDER.SN BSANTANDER.SN BSANTANDER.SN   
Date                                                                 
2023-01-03     29.868242     33.400002     33.950001     33.360001   
2023-01-04     29.555252     33.049999     33.450001     33.020000   
2023-01-05     29.760931     33.279999     33.340000     32.000000   
2023-01-06     29.921898     33.459999     33.459999     33.279999   
2023-01-09     30.118637     33.680000     33.750000     33.200001   

Price               Open        Volume  
Ticker     BSANTANDER.SN BSANTANDER.SN  
Date                                    
2023-01-03     33.400002       9392043  
2023-01-04     33.049999      14840191  
2023-01-05     33.279999       8638953  
2023-01-06     33.459999       1688317  
2023-01-09     33.680000      10631460  


In [ ]:
CHILE = yf.Ticker('CHILE.SN')
CHILE.info

{'address1': 'Ahumada 251',
 'city': 'Santiago',
 'country': 'Chile',
 'phone': '56 22 653 1111',
 'fax': '56 22 637 3434',
 'website': 'https://portales.bancochile.cl',
 'industry': 'Banks - Regional',
 'industryKey': 'banks-regional',
 'industryDisp': 'Banks - Regional',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': 'Banco de Chile, together with its subsidiaries, provides various banking services to customers in Chile. The company offers checking and debit accounts, debit and credit cards, and lines of credit; mortgage, consume, commercial, general purpose mortgage loans, and finance leases; and factoring services, mutual fund management, stock brokerage, foreign trade, payments and collections, insurance brokerage, including life and general insurance, as well as time deposits, savings instruments, and foreign currency services through branches under the Banco de Chile and Banco Edwards brands. It a

In [ ]:
hist=CHILE.history(period='max')
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2008-01-02 00:00:00-03:00,16.903757,16.903757,16.477045,16.477045,103887755,0.0,0.0
2008-01-03 00:00:00-03:00,16.477043,16.582665,16.265799,16.392546,34533343,0.0,0.0
2008-01-04 00:00:00-03:00,16.392547,16.392547,16.054557,16.160179,42583150,0.0,0.0
2008-01-07 00:00:00-03:00,16.181301,16.206649,16.012305,16.054554,36061655,0.0,0.0
2008-01-08 00:00:00-03:00,16.054554,16.227774,16.054554,16.054554,31888461,0.0,0.0


In [ ]:
CHILE.quarterly_balance_sheet.head()

,2024-09-30,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Ordinary Shares Number,101017081114.0,101017081114.0,101017081114.0,101017081114.0,101017081114.0,NaN,NaN
Share Issued,101017081114.0,101017081114.0,101017081114.0,101017081114.0,101017081114.0,NaN,NaN
Net Debt,9346872000000.0,7758090000000.0,7380157000000.0,7492046000000.0,6898674000000.0,NaN,NaN
Total Debt,12081401000000.0,11067952000000.0,11096130000000.0,10473679000000.0,10598972000000.0,NaN,NaN
Tangible Book Value,5321336000000.0,5188779000000.0,5030668000000.0,5911329000000.0,4898586000000.0,NaN,NaN


In [ ]:
CHILE.quarterly_cashflow.head()

,2024-09-30,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30
Free Cash Flow,-399036000000.0,-431346000000.0,-106772000000.0,50461000000.0,2105921000000.0,NaN
Repayment Of Debt,-287315000000.0,-325523000000.0,-275211000000.0,-687884000000.0,-552493000000.0,NaN
Issuance Of Debt,269382000000.0,208349000000.0,314872000000.0,287727000000.0,239595000000.0,NaN
Capital Expenditure,-16953000000.0,-18577000000.0,-20200000000.0,-22862000000.0,-23433000000.0,NaN
Interest Paid Supplemental Data,198649000000.0,260068000000.0,282370000000.0,321133000000.0,249539000000.0,NaN


In [ ]:
CHILE.quarterly_financials.head()

,2024-09-30,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Tax Effect Of Unusual Items,-131042712.439906,-438874921.255351,133206000.0,84268154.922001,-12773830.579805,NaN,NaN
Tax Rate For Calcs,0.216242,0.197425,0.2235,0.132706,0.26069,NaN,NaN
Total Unusual Items,-606000000.0,-2223000000.0,596000000.0,635000000.0,-49000000.0,NaN,NaN
Total Unusual Items Excluding Goodwill,-606000000.0,-2223000000.0,596000000.0,635000000.0,-49000000.0,NaN,NaN
Net Income From Continuing Operation Net Minority Interest,288071000000.0,323600000000.0,297655000000.0,515935000000.0,259993000000.0,NaN,NaN


In [ ]:
#hist.plot(kind='line', subplots=True)

De la web obtenemos los tickers del IPSA

In [ ]:
# Wikipedia URL for IPSA components
url = 'https://es.wikipedia.org/wiki/%C3%8Dndice_de_Precio_Selectivo_de_Acciones'

# Extract tables from the Wikipedia page
tables = pd.read_html(url)

# Display the first table (if it exists)
print(tables[0])

    N.º   Nemotécnico                             Empresa
0     1       AGUAS-A                       Aguas Andinas
1     2      ANDINA-B                Embotelladora Andina
2     3           BCI      Banco de Crédito e Inversiones
3     4    BSANTANDER               Banco Santander Chile
4     5           CAP                                 CAP
5     6           CCU      Compañía de Cervecerías Unidas
6     7    CENCOSHOPP              Cencosud Shopping S.A.
7     8      CENCOSUD                       Cencosud S.A.
8     9         CHILE                      Banco de Chile
9    10          CMPC                       Empresas CMPC
10   11        COLBUN                              Colbún
11   12    CONCHATORO                  Viña Concha y Toro
12   13         COPEC                               Copec
13   14           ECL                               Engie
14   15        ENELAM                       Enel Américas
15   16     ENELCHILE                          Enel Chile
16   17       

In [ ]:
df_idx=[i + '.SN' for i in tables[0]['Nemotécnico']]

check si cual de los tickers estan funcionando y filtramos los que no

In [ ]:
#check if the ticker are working

def validate_tickers(ticker_list):
    """
    Validate a list of tickers to check which ones are valid in yfinance.

    Parameters:
        ticker_list (list): List of tickers to validate.

    Returns:
        valid_tickers (list): A list of valid tickers.
        invalid_tickers (list): A list of invalid tickers.
    """
    valid_tickers = []
    invalid_tickers = []

    for ticker in ticker_list:
        try:
            # Attempt to fetch basic info for the ticker
            stock = yf.Ticker(ticker)

            # Try fetching a common piece of information like "info" or historical data
            if stock.info.get("bookValue") is None:
                print(f"Ticker {ticker} is invalid (no data in info).")
                invalid_tickers.append(ticker)
            else:
                #print(f"Ticker {ticker} is valid.")
                valid_tickers.append(ticker)
        except Exception as e:
            # Handle specific errors like 404 Client Errors
            print(f"An error occurred with ticker {ticker}: {e}")
            invalid_tickers.append(ticker)

    return valid_tickers, invalid_tickers


In [ ]:
#ticker list
ticker_list = df_idx

# Validate tickers
valid_tickers, invalid_tickers = validate_tickers(ticker_list)

print("\nValid Tickers:", valid_tickers)
print("Invalid Tickers:", invalid_tickers)

ERROR:yfinance:404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CENCOSHOPP.SN?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CENCOSHOPP.SN&crumb=0QuSVYzXJbO


Ticker CENCOSHOPP.SN is invalid (no data in info).
Ticker ORO%20BLANCO.SN is invalid (no data in info).

Valid Tickers: ['AGUAS-A.SN', 'ANDINA-B.SN', 'BCI.SN', 'BSANTANDER.SN', 'CAP.SN', 'CCU.SN', 'CENCOSUD.SN', 'CHILE.SN', 'CMPC.SN', 'COLBUN.SN', 'CONCHATORO.SN', 'COPEC.SN', 'ECL.SN', 'ENELAM.SN', 'ENELCHILE.SN', 'ENTEL.SN', 'FALABELLA.SN', 'IAM.SN', 'ITAUCL.SN', 'LTM.SN', 'MALLPLAZA.SN', 'PARAUCO.SN', 'QUINENCO.SN', 'RIPLEY.SN', 'SMU.SN', 'SONDA.SN', 'SQM-B.SN', 'VAPORES.SN']
Invalid Tickers: ['CENCOSHOPP.SN', 'ORO%20BLANCO.SN']


De los ticker filtrados obtenemos la informaciond de quarterly_financials, quarterly_cash_flow y quarterly_balance_sheet de los trimestres ['2024-06-30', '2024-03-31', '2023-12-31', '2023-09-30']

In [ ]:
def process_ticker_data(df_idx, ticker_attr, columns_to_keep):
    """
    Process ticker data based on the given attribute and append filtered data to a list.

    Parameters:
    - df_idx: List of ticker symbols.
    - ticker_attr: The attribute to extract from the ticker object (e.g., 'quarterly_balance_sheet').
    - columns_to_keep: List of column names to keep in the filtered DataFrame.

    Returns:
    - A list of filtered DataFrames for each ticker.
    """
    result_list = []

    # Iterate through a copy of df_idx to avoid modifying during iteration
    for item in df_idx[:]:
        try:
            # Get the ticker data
            l_ticker = yf.Ticker(item)

            # Dynamically access the attribute (e.g., quarterly_balance_sheet, quarterly_financials)
            quarterly_data = getattr(l_ticker, ticker_attr)

            # Filter columns
            filtered_df = quarterly_data[columns_to_keep]

            # Append to result list
            result_list.append(filtered_df)

            print(f"{item} columnas agregadas: {len(filtered_df.columns)}")

        except Exception as e:
            # Remove problematic tickers and log the error
            df_idx.remove(item)
            print(f"An error occurred in {item}: {e}")
            continue

    return result_list

#create list for quarterly_financials, quarterly_cash_flow,quarterly_balance_sheet
l_q_fs=[]
l_q_cf=[]
l_q_bs=[]

# Define shared parameters
columns_to_keep = ['2024-06-30', '2024-03-31', '2023-12-31', '2023-09-30']

# Process data for different ticker attributes
print('quarterly_financials\n')
if len(valid_tickers) != len(l_q_fs):
    l_q_fs = process_ticker_data(valid_tickers, 'quarterly_financials', columns_to_keep)

print('\nquarterly_cash_flow\n')
if len(valid_tickers) != len(l_q_cf):
    l_q_cf = process_ticker_data(valid_tickers, 'quarterly_cash_flow', columns_to_keep)

print('\nquarterly_balance_sheet\n')
if len(valid_tickers) != len(l_q_bs):
    l_q_bs = process_ticker_data(valid_tickers, 'quarterly_balance_sheet', columns_to_keep)


quarterly_financials

AGUAS-A.SN columnas agregadas: 4
ANDINA-B.SN columnas agregadas: 4
BCI.SN columnas agregadas: 4
BSANTANDER.SN columnas agregadas: 4
CAP.SN columnas agregadas: 4
CCU.SN columnas agregadas: 4
CENCOSUD.SN columnas agregadas: 4
CHILE.SN columnas agregadas: 4
CMPC.SN columnas agregadas: 4
COLBUN.SN columnas agregadas: 4
CONCHATORO.SN columnas agregadas: 4
COPEC.SN columnas agregadas: 4
ECL.SN columnas agregadas: 4
ENELAM.SN columnas agregadas: 4
ENELCHILE.SN columnas agregadas: 4
ENTEL.SN columnas agregadas: 4
FALABELLA.SN columnas agregadas: 4
IAM.SN columnas agregadas: 4
ITAUCL.SN columnas agregadas: 4
LTM.SN columnas agregadas: 4
MALLPLAZA.SN columnas agregadas: 4
PARAUCO.SN columnas agregadas: 4
QUINENCO.SN columnas agregadas: 4
RIPLEY.SN columnas agregadas: 4
SMU.SN columnas agregadas: 4
SONDA.SN columnas agregadas: 4
SQM-B.SN columnas agregadas: 4
VAPORES.SN columnas agregadas: 4

quarterly_cash_flow

AGUAS-A.SN columnas agregadas: 4
ANDINA-B.SN columnas agregada

In [ ]:
#remove NaN in the df list

def remove_nan_rows(dataframes_list):
    """
    Removes rows that contain only NaN values from each DataFrame in a list.

    Args:
    - dataframes_list (list): List of DataFrames to process.

    Returns:
    - list: List of DataFrames with rows containing only NaN removed.
    """
    updated_dataframes = []

    for i, df in enumerate(dataframes_list):
        try:
            # Drop rows with all NaN values
            df_cleaned = df.dropna(how="all")
            updated_dataframes.append(df_cleaned)
        except Exception as e:
            print(f"An error occurred at iteration {i}: {e}")
            updated_dataframes.append(df)  # Preserve the original DataFrame in case of error

    return updated_dataframes


In [ ]:
l_q_fs_cleaned = remove_nan_rows(l_q_fs)
l_q_cf_cleaned = remove_nan_rows(l_q_cf)
l_q_bs_cleaned = remove_nan_rows(l_q_bs)


In [ ]:
#consolidar las listas

# Consolidar las listas en una sola
# Consolidar los DataFrames correspondientes de cada lista y conservar los índices originales
lista_consolidada = [pd.concat([df1, df2, df3]) for df1, df2, df3 in zip(l_q_fs_cleaned, l_q_cf_cleaned, l_q_bs_cleaned)]


homogeneizaremos las muestras

In [ ]:
def contar_y_ordenar_indices(lista_dfs):
    from collections import Counter

    # Contador para acumular los índices de todos los DataFrames
    contador_indices = Counter()

    # Contar índices en cada DataFrame
    for df in lista_dfs:
        contador_indices.update(df.index)

    # Ordenar el contador de mayor a menor
    return dict(contador_indices.most_common())

# Ejemplo de uso:
# lista_consolidada = [df1, df2, df3, ...]
repeticiones_indices = contar_y_ordenar_indices(lista_consolidada)

# Ahora repeticiones_indices contiene el conteo total de cada índice, ordenado de mayor a menor


In [ ]:
repeticiones_indices = contar_y_ordenar_indices(lista_consolidada)
print(repeticiones_indices)

{'Tax Effect Of Unusual Items': 28, 'Tax Rate For Calcs': 28, 'Total Unusual Items': 28, 'Total Unusual Items Excluding Goodwill': 28, 'Net Income From Continuing Operation Net Minority Interest': 28, 'Net Interest Income': 28, 'Interest Expense': 28, 'Interest Income': 28, 'Normalized Income': 28, 'Net Income From Continuing And Discontinued Operation': 28, 'Diluted Average Shares': 28, 'Basic Average Shares': 28, 'Diluted EPS': 28, 'Basic EPS': 28, 'Diluted NI Availto Com Stockholders': 28, 'Net Income Common Stockholders': 28, 'Net Income': 28, 'Net Income Including Noncontrolling Interests': 28, 'Net Income Continuous Operations': 28, 'Tax Provision': 28, 'Pretax Income': 28, 'Special Income Charges': 28, 'Selling General And Administration': 28, 'General And Administrative Expense': 28, 'Total Revenue': 28, 'Operating Revenue': 28, 'Free Cash Flow': 28, 'Financing Cash Flow': 28, 'Investing Cash Flow': 28, 'Ordinary Shares Number': 28, 'Share Issued': 28, 'Total Debt': 28, 'Tangib

Por el criterio de 'Dividends Payable' que se repite 23 veces es que haremos un filtro, mantendremos aquellas que tengan ese criterio

In [ ]:
def filtrar_por_indice_y_posiciones(lista_dfs, indice):
    # Inicializar listas para los DataFrames filtrados y las posiciones eliminadas
    lista_filtrada = []
    posiciones_eliminadas = []

    # Recorrer los DataFrames con su índice en la lista
    for i, df in enumerate(lista_dfs):
        if indice in df.index:
            lista_filtrada.append(df)
        else:
            posiciones_eliminadas.append(i)

    return lista_filtrada, posiciones_eliminadas

# Ejemplo de uso:
# lista_consolidada = [df1, df2, df3, ...]
lista_consolidada_filtrada, posiciones_eliminadas = filtrar_por_indice_y_posiciones(lista_consolidada, 'Dividends Payable')

# Ahora:
# - lista_consolidada_filtrada contiene solo los DataFrames cuyo índice contiene 'Dividends Payable'
# - posiciones_eliminadas contiene las posiciones de los DataFrames que no cumplen la condición



In [ ]:
#Caules son los ticker descartados
for i in posiciones_eliminadas:
  print(df_idx[i])

BSANTANDER.SN
CONCHATORO.SN
COPEC.SN
ENTEL.SN
PARAUCO.SN


Pero vemos que existe un desbalance en el shape de los df creados, algunos tienen indices que otros no

In [ ]:
for i in lista_consolidada_filtrada:
  print(i.shape)
len(lista_consolidada_filtrada)

(168, 4)
(189, 4)
(145, 4)
(188, 4)
(196, 4)
(172, 4)
(154, 4)
(164, 4)
(167, 4)
(164, 4)
(183, 4)
(144, 4)
(163, 4)
(165, 4)
(166, 4)
(181, 4)
(154, 4)
(160, 4)
(196, 4)
(171, 4)
(178, 4)
(173, 4)
(138, 4)


23

Ahora mantendremos aquellos indices que sean comunes entre todos, para que todos tengan la misma estructura

In [ ]:
def conservar_filas_comunes_con_posiciones(lista_dfs):
    if not lista_dfs:  # Verificar si la lista está vacía
        return [], []

    # Encontrar la intersección de índices entre todos los DataFrames
    indices_comunes = set(lista_dfs[0].index)
    for df in lista_dfs[1:]:
        indices_comunes &= set(df.index)

    # Convertir los índices comunes a una lista
    indices_comunes = list(indices_comunes)

    # Filtrar DataFrames que contienen todos los índices comunes
    lista_filtrada = []
    posiciones_eliminados = []

    for i, df in enumerate(lista_dfs):
        if all(idx in df.index for idx in indices_comunes):
            lista_filtrada.append(df.loc[indices_comunes])
        else:
            posiciones_eliminados.append(i)

    return lista_filtrada, posiciones_eliminados

# Ejemplo de uso:
# lista_consolidada = [df1, df2, df3, ...]
lista_filtrada, posiciones_eliminados = conservar_filas_comunes_con_posiciones(lista_consolidada_filtrada)

# Ahora 'lista_filtrada' tiene los DataFrames filtrados
# Y 'posiciones_eliminados' tiene las posiciones de los DataFrames descartados

In [ ]:
posiciones_eliminados

[]

In [ ]:
for  i in lista_filtrada:
  print(i.shape)
print(len(lista_filtrada))

(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
(56, 4)
23


In [ ]:
def contar_nan_en_lista(lista_dfs):
    resultados = []
    for i, df in enumerate(lista_dfs):
        # Contar los NaN en todo el DataFrame
        num_nan = df.isna().sum().sum()
        # Agregar el resultado a la lista como tupla: (posición, número de NaN)
        resultados.append((i, num_nan))
    return resultados

# Ejemplo de uso:
# lista_consolidada = [df1, df2, df3, ...]
resultados_nan = contar_nan_en_lista(lista_filtrada)

# Ahora resultados_nan contiene las posiciones y el número de NaN en cada DataFrame


In [ ]:
resultados_nan

[(0, 4),
 (1, 0),
 (2, 4),
 (3, 4),
 (4, 0),
 (5, 0),
 (6, 4),
 (7, 4),
 (8, 8),
 (9, 4),
 (10, 8),
 (11, 45),
 (12, 4),
 (13, 4),
 (14, 4),
 (15, 6),
 (16, 7),
 (17, 4),
 (18, 4),
 (19, 4),
 (20, 4),
 (21, 4),
 (22, 13)]

In [ ]:
lista_filtrada[22]

,2024-06-30,2024-03-31,2023-12-31,2023-09-30
Total Equity Gross Minority Interest,6884833000.0,7922025000.0,8088551000.0,8084579000.0
Ordinary Shares Number,51319876188.0,51319876188.0,51319876188.0,51319876188.0
Diluted Average Shares,47941666667.0,51319876188.0,NaN,50273888889.0
Normalized Income,28765000.0,-159105000.0,1102000.0,-90493000.0
Total Capitalization,6884833000.0,7922025000.0,8088551000.0,8084579000.0
Total Revenue,0.0,0.0,0.0,0.0
Net Income From Continuing And Discontinued Operation,28765000.0,-159105000.0,1102000.0,-90493000.0
Basic Average Shares,47941666667.0,51319876188.0,NaN,50273888889.0
Total Unusual Items Excluding Goodwill,NaN,NaN,0.0,0.0
Share Issued,51319876188.0,51319876188.0,51319876188.0,51319876188.0


**-OBTENCION DE INDICADORES FINANCIEROS-**

-INDICADORES DE RENTABILIDAD

Margen Neto (Net Profit Margin):
Net Income/Total Revenue | LISTO

Margen Operativo (Operating Margin):
Operating Revenue/ Total Revenue | DESCARTADO, todos 1

ROE (Return on Equity):
Net Income/Stockholders Equity | LISTO

ROA (Return on Assets):
Net Income/Total Assets | LISTO

ROIC (Return on Invested Capital):
Net Income−Dividends / Invested Capital | DESCARTADO, problemas con dividends 3, 11, 16 , 23, 26

Earnings Yield:
EPS /Market Price per Share | POSPUESTO

Indicadores de Riesgo Financiero

    Deuda Neta a EBITDA:
    Net Debt (Total Debt - Cash And Cash Equivalents)/EBITDA |PROBLEMAS CON EBITDA PERO QUEDA
    
    Evalúa la capacidad de la empresa para pagar su deuda neta usando sus ganancias operativas.

    Ratio de Apalancamiento (Leverage Ratio):
    Total Liabilities/Total Equity | LISTO
    
    Indica cuánto riesgo financiero asume la empresa al financiarse con deuda.

In [ ]:
#ULTIMO Y ACTUAL
def calculate_metric(data_list_fs, data_list_bs, numerator_key, denominator_key, result_key, numerator_subtract_key=None, data_list_subtract=None):
    results = []
    for ele in range(len(data_list_fs)):
        try:
            # Handle the numerator subtraction if specified
            if numerator_subtract_key and data_list_subtract:
                numerator = data_list_fs[ele].loc[numerator_key] - data_list_subtract[ele].loc[numerator_subtract_key]
            else:
                numerator = data_list_fs[ele].loc[numerator_key]

            # Denominator
            denominator = data_list_bs[ele].loc[denominator_key]

            # Check if the denominator contains any zero values
            if (denominator == 0).any():
                # Create a DataFrame with all values set to 0
                result = pd.DataFrame(
                    0,
                    columns=data_list_fs[ele].columns,
                    index=[result_key]
                )
            else:
                # Perform the calculation
                result = pd.DataFrame(
                    [numerator / denominator],
                    columns=data_list_fs[ele].columns,
                    index=[result_key]
                )

            # Append the result DataFrame to the results list
            results.append(result)

        except Exception as e:
            print(f"An error occurred for element {ele}: {e}")
            continue

    return results


In [ ]:
#without sustraction
results = calculate_metric(l_q_fs, l_q_bs, 'Net Income', 'Total Assets', 'Return on Assets')

#with sustraction
results = calculate_metric(l_q_fs, l_q_bs, 'Net Income', 'Other Expenses', 'Return on Assets', numerator_subtract_key='Taxes', data_list_subtract=l_q_fs)



In [ ]:
#Margen Neto (Net Profit Margin): -OK-
#l_q_fs[10].loc['Net Income']/l_q_fs[10].loc['Total Revenue']

results = calculate_metric(l_q_fs, l_q_fs, 'Net Income', 'Total Revenue', 'Margen Neto')

In [ ]:
#ROE (Return on Equity): Net Income/Stockholders Equity -OK-
#l_q_fs[ele].loc['Net Income']/l_q_bs[ele].loc['Stockholders Equity']

results = calculate_metric(l_q_fs, l_q_bs, 'Net Income', 'Stockholders Equity', 'Return on Equity')

In [ ]:
#ROA (Return on Assets): Net Income/Total Assets -OK-
#l_q_fs[ele].loc['Net Income']/l_q_bs[ele].loc['Total Assets']

results = calculate_metric(l_q_fs, l_q_bs, 'Net Income', 'Total Assets', 'Return on Assets')

In [ ]:
#ROIC (Return on Invested Capital): Net Income − Dividends / Invested Capital
#l_q_fs[ele].loc['Net Income'] - l_q_bs[ele].loc['Dividends Payable'] / l_q_bs[ele].loc['Invested Capital']

results = calculate_metric(l_q_fs, l_q_bs, 'Net Income', 'Dividends Payable', 'Return on Invested Capital', numerator_subtract_key='Dividends Payable', data_list_subtract=l_q_bs)


An error occurred for element 3: 'Dividends Payable'
An error occurred for element 11: 'Dividends Payable'
An error occurred for element 12: 'Dividends Payable'
An error occurred for element 16: 'Dividends Payable'
An error occurred for element 23: 'Dividends Payable'


In [ ]:
#Deuda Neta a EBITDA: Net Debt (Total Debt - Cash And Cash Equivalents)/EBITDA
#(l_q_bs[0].loc['Total Debt'] - l_q_bs[0].loc['Cash And Cash Equivalents'] / l_q_fs[0].loc['EBITDA']

results = calculate_metric(l_q_bs, l_q_fs, 'Total Debt', 'EBITDA', 'EBITDA: Net Debt', numerator_subtract_key='Cash And Cash Equivalents', data_list_subtract=l_q_bs)


An error occurred for element 2: 'EBITDA'
An error occurred for element 3: 'EBITDA'
An error occurred for element 7: 'EBITDA'
An error occurred for element 18: 'EBITDA'


In [ ]:
l_q_fs[2].loc['EBITDA']

KeyError: 'EBITDA'

In [ ]:
#Earnings Yield: EPS /Market Price per Share
#l_q_fs[ele].loc['Diluted EPS']/l_q_bs[ele].loc['Total Assets']

In [ ]:
#Ratio de Apalancamiento (Leverage Ratio): Total Liabilities/Total Equity -OK-
#Indica cuánto riesgo financiero asume la empresa al financiarse con deuda
#l_q_bs[ele].loc['Total Liabilities Net Minority Interest']/l_q_bs[ele].loc['Total Assets']

results = calculate_metric(l_q_bs, l_q_bs, 'Total Liabilities Net Minority Interest', 'Total Assets', 'Leverage Ratio')

2. Indicadores de ValuacióN

Precio/Ganancia (P/E Ratio):
EPS/Market Price per Share​

Precio/Valor en Libros (P/B Ratio):
Market Price per Share / (Stockholders Equity/Shares Outstanding)

Enterprise Value/EBITDA (EV/EBITDA):
Enterprise Value​ / EBITDA

Precio/Ventas (P/S Ratio):
Market Capitalization / Total Revenue


In [ ]:
from sklearn.impute import KNNImputer

def knn_impute_nan_entire_df(dataframes_list, n_neighbors=3):
    """
    Use KNN to impute NaN values in a list of DataFrames for all columns.

    Args:
    - dataframes_list (list): List of DataFrames to process.
    - n_neighbors (int): Number of neighbors to use for KNN imputation.

    Returns:
    - list: List of DataFrames with NaN values imputed.
    """
    updated_dataframes = []

    for i, df in enumerate(dataframes_list):
        try:
            # Transpose the DataFrame to match KNNImputer's expected input format
            df_transposed = df.T

            # Apply KNN imputer
            imputer = KNNImputer(n_neighbors=n_neighbors)
            imputed_data = imputer.fit_transform(df_transposed)

            # Reconstruct the DataFrame with imputed values
            df_imputed = pd.DataFrame(imputed_data, columns=df_transposed.columns, index=df_transposed.index).T

            # Append the updated DataFrame to the list
            updated_dataframes.append(df_imputed)
        except Exception as e:
            print(valid_tickers[i])
            print(f"An error occurred at iteration {i} (DataFrame index {i}): {e}")
            updated_dataframes.append(df)

    return updated_dataframes

# Apply KNN imputation to the entire DataFrame
l_q_fs_knn_imputed_all = knn_impute_nan_entire_df(l_q_fs_cleaned, n_neighbors=3)

# Check the imputed DataFrames
#for i, df in enumerate(l_q_fs_knn_imputed_all):
#    print(f"Imputed DataFrame {i}:")
#    print(df)


In [ ]:
valid_tickers[18]

'ITAUCL.SN'